# File I/O

So far we have discussed how to process data and how
to build, train, and test deep learning models.
However, at some point we will hopefully be happy enough
with the learned models that we will want
to save the results for later use in various contexts
(perhaps even to make predictions in deployment).
Additionally, when running a long training process,
the best practice is to periodically save intermediate results (checkpointing)
to ensure that we do not lose several days' worth of computation
if we trip over the power cord of our server.
Thus it is time to learn how to load and store
both individual weight vectors and entire models.
This section addresses both issues.


In [1]:
import torch
from torch import nn
from torch.nn import functional as F

## (**Loading and Saving Tensors**)

For individual tensors, we can directly
invoke the `load` and `save` functions
to read and write them respectively.
Both functions require that we supply a name,
and `save` requires as input the variable to be saved.


In [3]:
x = torch.arange(4)
torch.save(x, 'x-file')
x

tensor([0, 1, 2, 3])

We can now read the data from the stored file back into memory.


In [4]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

We can [**store a list of tensors and read them back into memory.**]


In [5]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

We can even [**write and read a dictionary that maps
from strings to tensors.**]
This is convenient when we want
to read or write all the weights in a model.


In [15]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [13]:
import numpy as np
np.savetxt('comotxtnumpy.txt', x.numpy())

pp = torch.rand(size=(2,2,2))
pp.numpy()
# np.savetxt('ppnumpy.txt', pp.numpy())


array([[[0.6621208 , 0.6881579 ],
        [0.31823045, 0.37319767]],

       [[0.27871293, 0.56241566],
        [0.05610281, 0.4356488 ]]], dtype=float32)

## [**Loading and Saving Model Parameters**]

Saving individual weight vectors (or other tensors) is useful,
but it gets very tedious if we want to save
(and later load) an entire model.
After all, we might have hundreds of
parameter groups sprinkled throughout.
For this reason the deep learning framework provides built-in functionalities
to load and save entire networks.
An important detail to note is that this
saves model *parameters* and not the entire model.
For example, if we have a 3-layer MLP,
we need to specify the architecture separately.
The reason for this is that the models themselves can contain arbitrary code,
hence they cannot be serialized as naturally.
Thus, in order to reinstate a model, we need
to generate the architecture in code
and then load the parameters from disk.
(**Let's start with our familiar MLP.**)


In [18]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.output = nn.LazyLinear(10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [19]:
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[-0.1567,  0.0420, -0.1314,  ..., -0.1994,  0.1179,  0.0318],
                      [-0.1015,  0.1731,  0.2232,  ...,  0.1567,  0.1946, -0.1466],
                      [-0.0523,  0.1989, -0.1097,  ...,  0.2090, -0.0789,  0.0531],
                      ...,
                      [ 0.0680,  0.0329,  0.1696,  ...,  0.0896,  0.0148,  0.1942],
                      [ 0.0760, -0.1721, -0.1273,  ...,  0.1425,  0.0643, -0.2060],
                      [-0.0346,  0.1174, -0.1435,  ..., -0.1155,  0.0590, -0.1074]])),
             ('hidden.bias',
              tensor([ 4.5742e-02,  8.9421e-02, -2.2209e-01,  1.0356e-01, -5.3999e-02,
                      -2.8820e-02,  1.1388e-01,  1.1602e-01, -1.9682e-01,  7.3428e-02,
                      -1.2763e-02, -1.1967e-01, -9.1832e-02, -1.7084e-01,  2.9659e-02,
                      -2.2017e-01, -4.9351e-02, -9.3244e-02,  7.5740e-02, -1.7409e-01,
                       3.4471e-02, -1.1606e-01,  1.0633e-0

Next, we [**store the parameters of the model as a file**] with the name "mlp.params".


In [20]:
torch.save(net.state_dict(), 'mlp.params')

To recover the model, we instantiate a clone
of the original MLP model.
Instead of randomly initializing the model parameters,
we [**read the parameters stored in the file directly**].


In [21]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

Since both instances have the same model parameters,
the computational result of the same input `X` should be the same.
Let's verify this.


In [22]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [23]:
# What about nn.Sequential?

## Summary

The `save` and `load` functions can be used to perform file I/O for tensor objects.
We can save and load the entire sets of parameters for a network via a parameter dictionary.
Saving the architecture has to be done in code rather than in parameters.

## Exercises

1. Even if there is no need to deploy trained models to a different device, what are the practical benefits of storing model parameters?
1. Assume that we want to reuse only parts of a network to be incorporated into a network having a different architecture. How would you go about using, say the first two layers from a previous network in a new network?
1. How would you go about saving the network architecture and parameters? What restrictions would you impose on the architecture?


[Discussions](https://discuss.d2l.ai/t/61)
